In [ ]:
import pandas as pd
import torch
import os
import numpy as np
import time
import optuna # Para HPO
import traceback # Para imprimir errores detallados

# --- PyKeen Imports ---
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline # Para el entrenamiento final
from pykeen.hpo import hpo_pipeline  # Para la optimización
# Importar modelos (aunque hpo_pipeline puede usar strings)
from pykeen.models import TransE, DistMult, ComplEx, RotatE

optuna.logging.set_verbosity(optuna.logging.WARNING)

print("Imports completados.")

/home/javimc/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports completados.


In [ ]:
# --- Configuración General ---
USE_SMALL_DATASET = False # USAR DATASET COMPLETO

# --- Rutas a los archivos ---
FULL_CSV_PATH = 'new_triplets20_optimized.csv'
# Asegúrate de que la ruta sea correcta respecto a tu notebook
SMALL_CSV_PATH = 'small_triplets.csv' # No se usará si USE_SMALL_DATASET = False

# --- Directorio de Resultados ---
# Usar un nombre descriptivo para esta ejecución completa con HPO
RESULTS_DIR = 'pykeen_full_hpo_results'

# --- Semilla para Reproducibilidad ---
SEED = 42
# Fijar semillas para PyTorch y NumPy
torch.manual_seed(SEED)
np.random.seed(SEED)
# Nota: La reproducibilidad total en CUDA a veces requiere más configuraciones

# --- Configuración de HPO (Optimización de Hiperparámetros) ---
# Número de pruebas HPO a ejecutar POR MODELO
N_HPO_TRIALS = 30 # Puedes ajustar esto (más pruebas = más tiempo)

# Métrica a optimizar y dirección
HPO_METRIC = 'hits_at_10' # Métrica clave para recomendación/ranking
HPO_DIRECTION = 'maximize'

# Modelos a incluir en la búsqueda HPO
MODELS_FOR_HPO = ['TransE', 'DistMult', 'RotatE'] # Lista de modelos a comparar

# --- Configuración de Entrenamiento FINAL (después de HPO) ---
# Número máximo de épocas para el entrenamiento final del mejor modelo.
# Early Stopping decidirá las épocas reales.
FINAL_TRAINING_EPOCHS = 200 # Un número razonablemente alto

# --- Configuración de Inferencia (igual que antes) ---
TARGET_INGREDIENT = "potato"
TARGET_CALORIE_LEVEL = "high_calories"
INGREDIENT_RELATION = "has_ingredient"
CALORIE_RELATION = "has_calories"

print("Configuración definida.")
print(f"Usando dataset completo: {not USE_SMALL_DATASET}")
print(f"Optimizando para: {HPO_METRIC} ({HPO_DIRECTION})")
print(f"Número de pruebas HPO por modelo: {N_HPO_TRIALS}")
print(f"Modelos para HPO: {MODELS_FOR_HPO}")

Configuración definida.
Usando dataset completo: True
Optimizando para: hits_at_10 (maximize)
Número de pruebas HPO por modelo: 30
Modelos para HPO: ['TransE', 'DistMult', 'RotatE']


In [3]:
# --- Determinar qué archivo usar ---
if USE_SMALL_DATASET:
    CSV_PATH = SMALL_CSV_PATH
    print(f"--- USANDO DATASET PEQUEÑO: {CSV_PATH} ---")
else:
    CSV_PATH = FULL_CSV_PATH
    print(f"--- USANDO DATASET COMPLETO: {CSV_PATH} ---")

# Crear directorio de resultados si no existe
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Comprobación de existencia del archivo seleccionado ---
if not os.path.exists(CSV_PATH):
    print(f"¡ERROR! El archivo CSV seleccionado no se encontró en: {CSV_PATH}")
    raise FileNotFoundError(f"Archivo no encontrado: {CSV_PATH}")
else:
    print(f"Archivo encontrado: {CSV_PATH}")

# --- Comprobar Disponibilidad de CUDA (GPU) y definir dispositivo ---
if torch.cuda.is_available():
    try:
        gpu_name = torch.cuda.get_device_name(0)
        print(f"\nCUDA (GPU) disponible! Se intentará usar: {gpu_name}")
        torch.cuda.empty_cache()
        TARGET_DEVICE = 'cuda'
    except Exception as e_cuda:
        print(f"\nCUDA parece disponible pero hubo un error al obtener info ({e_cuda}). Usando CPU.")
        TARGET_DEVICE = 'cpu'
else:
    print("\nCUDA (GPU) no disponible. Usando CPU.")
    TARGET_DEVICE = 'cpu'

# --- Inicializar variables globales para las celdas ---
tf = None
training_factory, validation_factory, testing_factory = None, None, None
load_successful = False
# Para HPO
all_hpo_results = {}
best_overall_study = None
best_overall_value = -float('inf') if HPO_DIRECTION == 'maximize' else float('inf')
best_model_name_from_hpo = None
best_params = None # <--- Guardará los parámetros optimizados
# Para Entrenamiento Final
best_pipeline_result = None
final_metrics_available = False
# Para Resumen Final
final_summary_df = pd.DataFrame()

print(f"Dispositivo objetivo para entrenamiento/HPO: {TARGET_DEVICE}")

--- USANDO DATASET COMPLETO: new_triplets20_optimized.csv ---
Archivo encontrado: new_triplets20_optimized.csv

CUDA (GPU) disponible! Se intentará usar: NVIDIA GeForce RTX 4070 Laptop GPU
Dispositivo objetivo para entrenamiento/HPO: cuda


In [4]:
# --- Carga y División de Datos ---
print("\n--- Iniciando Carga y División de Datos ---")
start_load_time = time.time()
try:
    # --- Carga con Pandas + from_labeled_triples ---
    print(f"Cargando datos con Pandas desde {CSV_PATH}...")
    # Especificar low_memory=False puede ayudar con datasets grandes y tipos mixtos
    df_manual = pd.read_csv(CSV_PATH, dtype=str, header=0, low_memory=False)
    if df_manual.shape[1] != 3:
        raise ValueError(f"Pandas no detectó 3 columnas en {CSV_PATH}")
    # Limpiar posibles espacios extra en etiquetas (importante para mapeo)
    df_manual = df_manual.map(lambda x: x.strip() if isinstance(x, str) else x)
    print(f"-> {len(df_manual)} filas cargadas por Pandas.")

    labeled_triples_array = df_manual[['head', 'relation', 'tail']].to_numpy()
    print(f"Datos en array NumPy (shape {labeled_triples_array.shape}), usando from_labeled_triples...")

    # Crear TriplesFactory
    tf = TriplesFactory.from_labeled_triples(
        triples=labeled_triples_array,
        create_inverse_triples=True,
    )
    print("¡Carga con TriplesFactory exitosa!")

    # --- División ---
    print("Dividiendo ternas (80% train, 10% validation, 10% test)...")
    training_factory, validation_factory, testing_factory = tf.split(
        ratios=[0.8, 0.1, 0.1], random_state=SEED
    )
    print("División completada.")
    load_successful = True

    # --- Estadísticas ---
    print(f"\nNúmero de entidades únicas: {tf.num_entities}")
    print(f"Número de relaciones únicas (incl. inversas): {tf.num_relations}")
    print(f"Número total de ternas cargadas (incl. inversas): {tf.num_triples}")
    print(f"  -> Ternas de entrenamiento: {training_factory.num_triples}")
    print(f"  -> Ternas de validación (para HPO/EarlyStopping): {validation_factory.num_triples}")
    print(f"  -> Ternas de prueba (evaluación final): {testing_factory.num_triples}")

except Exception as load_err:
    print(f"\nERROR FATAL durante la carga o división: {type(load_err).__name__}: {load_err}")
    traceback.print_exc() # Imprimir traceback completo
    load_successful = False # Asegurar que esté False

end_load_time = time.time()
if load_successful:
    print(f"Tiempo de carga y división: {end_load_time - start_load_time:.2f} segundos")


--- Iniciando Carga y División de Datos ---
Cargando datos con Pandas desde new_triplets20_optimized.csv...
-> 141223 filas cargadas por Pandas.
Datos en array NumPy (shape (141223, 3)), usando from_labeled_triples...
¡Carga con TriplesFactory exitosa!
Dividiendo ternas (80% train, 10% validation, 10% test)...
División completada.

Número de entidades únicas: 19311
Número de relaciones únicas (incl. inversas): 16
Número total de ternas cargadas (incl. inversas): 135907
  -> Ternas de entrenamiento: 108725
  -> Ternas de validación (para HPO/EarlyStopping): 13591
  -> Ternas de prueba (evaluación final): 13591
Tiempo de carga y división: 0.26 segundos


In [14]:
# Celda 5: Optimización de Hiperparámetros (HPO) - CORREGIDA (Extracción de best_params)

# Verificar que la carga fue exitosa antes de proceder
if load_successful:
    print("\n--- Iniciando Optimización de Hiperparámetros (Iterando por Modelo) ---")
    hpo_overall_start_time = time.time()

    # Iterar sobre cada modelo a probar
    for model_name in MODELS_FOR_HPO:
        print(f"\n=================================================")
        print(f"--- Iniciando HPO para el modelo: {model_name} ---")
        print(f"=================================================")
        print(f"Optimizando para '{HPO_METRIC}' ({HPO_DIRECTION}) con {N_HPO_TRIALS} pruebas en device '{TARGET_DEVICE}'.")
        hpo_model_start_time = time.time()
        current_hpo_result = None # Resultado para este modelo

        try:
            # Ejecutar hpo_pipeline PARA ESTE MODELO específico
            current_hpo_result = hpo_pipeline(
                n_trials=N_HPO_TRIALS,
                training=training_factory,
                validation=validation_factory,
                testing=testing_factory, # Pasado para consistencia, evaluación principal en validación
                model=model_name,

                # --- Espacio de Búsqueda ---
                model_kwargs_ranges=dict(
                    embedding_dim=dict(type='int', low=64, high=512, step=64), # Ajustado rango dim
                ),
                training_kwargs_ranges=dict(
                    batch_size=dict(type='categorical', choices=[512, 1024, 2048]), # Ajustado para GPU
                ),
                optimizer_kwargs_ranges=dict(
                     lr=dict(type='float', low=5e-5, high=5e-3, log=True), # Ajustado rango lr
                ),
                regularizer_kwargs_ranges=dict(
                    # Optuna puede prefijar con el nombre del regularizador, ej 'LpRegularizer.weight'
                    # O simplemente 'weight'. Probamos con 'weight' por simplicidad.
                    weight=dict(type="float", low=1e-6, high=0.1, log=True), # Rango amplio para peso
                    # p=dict(type="categorical", choices=[1, 2]) # Podríamos optimizar L1 vs L2
                ) if model_name in ['DistMult', 'ComplEx', 'RotatE'] else None, # Solo para modelos que lo usan
                 # --- Fin Espacio de Búsqueda ---

                metric=HPO_METRIC,
                direction=HPO_DIRECTION,
                sampler='TPE', # Algoritmo de muestreo de Optuna

                # Early Stopping dentro de cada prueba HPO
                stopper='early',
                stopper_kwargs=dict(
                    frequency=10,       # Reducir frecuencia de evaluación en HPO
                    patience=5,         # Aumentar un poco la paciencia
                    metric=HPO_METRIC,
                    larger_is_better=(HPO_DIRECTION == 'maximize'),
                ),

                # Usar GPU si está disponible
                device=TARGET_DEVICE,

                # Especificar batch size para evaluación (puede ser mayor)
                evaluation_kwargs = dict(batch_size=2048), # Aumentado para GPU
            )

            hpo_model_end_time = time.time()
            print(f"--- HPO para {model_name} Completado ---")
            print(f"Tiempo de HPO para {model_name}: {(hpo_model_end_time - hpo_model_start_time)/60:.2f} min")

            # Guardar el resultado de HPO para este modelo en memoria
            all_hpo_results[model_name] = current_hpo_result
            print(f"Resultados de HPO para {model_name} almacenados en memoria.")

            # Comparar con el mejor resultado general encontrado hasta ahora
            if hasattr(current_hpo_result, 'study') and hasattr(current_hpo_result.study, 'best_value') and current_hpo_result.study.best_value is not None:
                current_best_value = current_hpo_result.study.best_value
                print(f"Mejor valor de '{HPO_METRIC}' para {model_name}: {current_best_value:.4f}")

                update_overall_best = False
                if best_overall_study is None: # Primera vez
                     update_overall_best = True
                elif HPO_DIRECTION == 'maximize' and current_best_value > best_overall_value:
                     update_overall_best = True
                elif HPO_DIRECTION == 'minimize' and current_best_value < best_overall_value:
                     update_overall_best = True

                if update_overall_best:
                    print(f"*** Nuevo mejor resultado general encontrado con {model_name}! ***")
                    best_overall_value = current_best_value
                    best_overall_study = current_hpo_result.study # Guardamos el estudio de Optuna
                    best_model_name_from_hpo = model_name         # Guardamos el nombre del modelo
            else:
                print(f"Advertencia: No se pudo obtener el best_value del estudio para {model_name}.")


        except Exception as hpo_err:
            print(f"\nERROR FATAL durante HPO para {model_name}: {type(hpo_err).__name__}: {hpo_err}")
            traceback.print_exc() # Imprimir detalles del error
            continue # Continuar con el siguiente modelo

    # --- Fin del bucle HPO por modelo ---
    hpo_overall_end_time = time.time()
    print(f"\n--- Optimización HPO General Completada ---")
    print(f"Tiempo total de HPO para todos los modelos: {(hpo_overall_end_time - hpo_overall_start_time)/60:.2f} minutos")

    # --- Extraer y Mostrar el Mejor Modelo y *Parámetros Optimizados* ---
    best_params = None # Inicializar
    if best_model_name_from_hpo and best_overall_study:
        print(f"\nMejor modelo general encontrado: {best_model_name_from_hpo}")
        print(f"Mejor valor de '{HPO_METRIC}' en validación: {best_overall_value:.4f}")

        # --- Extracción de Parámetros Optimizados ---
        print("\nIntentando extraer los parámetros optimizados de best_trial.params...")
        try:
            if hasattr(best_overall_study, 'best_trial') and best_overall_study.best_trial and hasattr(best_overall_study.best_trial, 'params'):
                 best_params = best_overall_study.best_trial.params # <-- Extraer PARAMS
                 print("Mejores Hiperparámetros optimizados (extraídos exitosamente):")
                 print(best_params) # Imprimir el diccionario de parámetros
            else:
                 print("Error: No se pudo acceder a best_trial.params.")
                 best_params = None
                 best_model_name_from_hpo = None # Marcar como fallido
        except Exception as e_params:
             print(f"Error al extraer parámetros de best_trial.params: {e_params}")
             best_params = None
             best_model_name_from_hpo = None # Marcar como fallido
        # --- FIN Extracción ---

    else:
        print("\nNo se pudo completar exitosamente la HPO o no se encontró un mejor modelo.")
        best_params = None
        best_model_name_from_hpo = None

else:
    print("\n--- La carga/división de datos falló. Saltando HPO. ---")

No random seed is specified. Setting to 1979955036.



--- Iniciando Optimización de Hiperparámetros (Iterando por Modelo) ---

--- Iniciando HPO para el modelo: TransE ---
Optimizando para 'hits_at_10' (maximize) con 30 pruebas en device 'cuda'.


INFO:pykeen.triples.triples_factory:Creating inverse triples.
Training epochs on cuda:0:   2%|▏         | 9/500 [00:25<20:58,  2.56s/epoch, loss=0.289, prev_loss=0.29] INFO:pykeen.evaluation.evaluator:Evaluation took 7.60s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 10: 0.33279376057685234. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-2f96eb70-10f6-4a36-90ab-a0c74ca4b745.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
Training epochs on cuda:0:  12%|█▏        | 59/500 [03:12<19:37,  2.67s/epoch, loss=0.286, prev_loss=0.286]INFO:pykeen.evaluation.evaluator:Evaluation took 7.54s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 60: 0.34795084982709146. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-2f96eb70-10f6-4a36-90ab-a0c74ca4b745.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 60.
Training epoc

--- HPO para TransE Completado ---
Tiempo de HPO para TransE: 49.73 min
Resultados de HPO para TransE almacenados en memoria.
Mejor valor de 'hits_at_10' para TransE: 0.5005
*** Nuevo mejor resultado general encontrado con TransE! ***

--- Iniciando HPO para el modelo: DistMult ---
Optimizando para 'hits_at_10' (maximize) con 30 pruebas en device 'cuda'.


Training epochs on cuda:0:   2%|▏         | 9/400 [00:26<17:19,  2.66s/epoch, loss=1.4, prev_loss=1.5]  INFO:pykeen.evaluation.evaluator:Evaluation took 12.69s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 10: 0.3993083658303289. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-7c6b567f-bd66-4331-a34e-57e017135edd.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
Training epochs on cuda:0:   5%|▍         | 19/400 [01:05<17:43,  2.79s/epoch, loss=0.771, prev_loss=0.792]INFO:pykeen.evaluation.evaluator:Evaluation took 12.62s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 20: 0.41520123611213305. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-7c6b567f-bd66-4331-a34e-57e017135edd.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 20.
Training epochs on cuda:0:  17%|█▋        | 69/400 [04:19<15:20,  2.78s/epoc

--- HPO para DistMult Completado ---
Tiempo de HPO para DistMult: 64.61 min
Resultados de HPO para DistMult almacenados en memoria.
Mejor valor de 'hits_at_10' para DistMult: 0.4188

--- Iniciando HPO para el modelo: RotatE ---
Optimizando para 'hits_at_10' (maximize) con 30 pruebas en device 'cuda'.


Training epochs on cuda:0:   1%|          | 9/800 [00:17<22:46,  1.73s/epoch, loss=0.655, prev_loss=0.662]INFO:pykeen.evaluation.evaluator:Evaluation took 7.64s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 10: 0.3189978662350085. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-366f427d-9bba-4e6a-8649-4ee786daac05.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
Training epochs on cuda:0:   2%|▏         | 19/800 [00:42<23:29,  1.80s/epoch, loss=0.552, prev_loss=0.561]INFO:pykeen.evaluation.evaluator:Evaluation took 7.61s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 20: 0.39941873298506364. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-366f427d-9bba-4e6a-8649-4ee786daac05.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 20.
Training epochs on cuda:0:   9%|▊         | 69/800 [02:45<21:53,  1.80s/epoc

--- HPO para RotatE Completado ---
Tiempo de HPO para RotatE: 148.41 min
Resultados de HPO para RotatE almacenados en memoria.
Mejor valor de 'hits_at_10' para RotatE: 0.4221

--- Optimización HPO General Completada ---
Tiempo total de HPO para todos los modelos: 262.74 minutos

Mejor modelo general encontrado: TransE
Mejor valor de 'hits_at_10' en validación: 0.5005

Intentando extraer los parámetros optimizados de best_trial.params...
Mejores Hiperparámetros optimizados (extraídos exitosamente):
{'model.embedding_dim': 384, 'model.scoring_fct_norm': 2, 'loss.margin': 1.8213771944268697, 'optimizer.lr': 0.0003184982126337148, 'negative_sampler.num_negs_per_pos': 6, 'training.num_epochs': 1000, 'training.batch_size': 512}


In [15]:
# Celda 6: Entrenamiento Final del Mejor Modelo - CORREGIDA (Reconstrucción de Config)

# --- Entrenamiento Final del Mejor Modelo (Solo si HPO fue exitoso y encontró parámetros) ---
best_pipeline_result = None # Inicializar

# Verificar si HPO fue exitoso y encontró parámetros válidos
# Comprobar que las variables de la celda anterior existen y son válidas
if ('load_successful' in locals() and load_successful and
    'best_model_name_from_hpo' in locals() and best_model_name_from_hpo is not None and
    'best_params' in locals() and isinstance(best_params, dict)): # <-- Comprobar si best_params es un diccionario

    print(f"\n--- Iniciando Entrenamiento Final con el Mejor Modelo y Parámetros ---")
    print(f"Modelo Seleccionado por HPO: {best_model_name_from_hpo}")
    print(f"Device para Entrenamiento Final: '{TARGET_DEVICE}'")
    print(f"Parámetros Optimizados a usar: {best_params}")
    final_train_start_time = time.time()

    try:
        # --- Reconstruir argumentos para pipeline desde best_params ---
        # Diccionarios para guardar los kwargs reconstruidos
        model_kwargs_final = {}
        loss_kwargs_final = {}
        optimizer_kwargs_final = {}
        training_kwargs_final = {}
        negative_sampler_kwargs_final = {}
        regularizer_final = None
        regularizer_kwargs_final = {}

        # --- Extraer parámetros optimizados (manejando prefijos de Optuna) ---
        # Iterar sobre los parámetros encontrados por Optuna
        for param_key, param_value in best_params.items():
            parts = param_key.split('.')
            # Asumimos estructura 'componente.parametro' o 'componente.subcomponente.parametro'
            category = parts[0]
            name = parts[-1] # El nombre real del hiperparámetro

            if category == 'model':
                # Extraer parámetros del modelo (ej. embedding_dim, scoring_fct_norm)
                # Optuna puede o no usar el prefijo 'model.'
                actual_model_param_name = '.'.join(parts[1:]) if len(parts)>1 else name
                model_kwargs_final[actual_model_param_name] = param_value
            elif category == 'loss':
                # Extraer parámetros de la función de pérdida (ej. margin)
                loss_kwargs_final[name] = param_value
            elif category == 'optimizer':
                # Extraer parámetros del optimizador (ej. lr)
                optimizer_kwargs_final[name] = param_value
            elif category == 'training':
                 # Extraer parámetros del entrenamiento (ej. batch_size)
                 # El número de épocas de HPO no lo usamos aquí, usamos FINAL_TRAINING_EPOCHS
                 if name != 'num_epochs':
                     training_kwargs_final[name] = param_value
            elif category == 'negative_sampler':
                # Extraer parámetros del muestreador negativo (ej. num_negs_per_pos)
                negative_sampler_kwargs_final[name] = param_value
            elif category == 'regularizer':
                 # Asumir LpRegularizer si se optimizó 'weight'
                 if name == 'weight':
                     regularizer_final = 'LpRegularizer'
                     regularizer_kwargs_final[name] = param_value
                     regularizer_kwargs_final.setdefault('p', 2) # Default a L2 si 'p' no se optimizó
                 else: # Guardar otros posibles params del regularizador
                     regularizer_kwargs_final[name] = param_value

        # --- Asignar componentes Fijos (basados en defaults de PyKeen/logs HPO) ---
        # Estos no fueron optimizados directamente por nombre, usamos los que PyKeen eligió
        # (Asegúrate de que coinciden con los logs si quieres ser preciso)
        optimizer_final = 'Adam'
        # Asumir loss basada en parámetros encontrados o modelo (TransE -> MarginRankingLoss)
        loss_final = 'MarginRankingLoss' if 'margin' in loss_kwargs_final else 'NLLLoss' # Ajusta si es necesario
        # Asumir sampler basado en parámetros encontrados
        negative_sampler_final = 'BasicNegativeSampler' if 'num_negs_per_pos' in negative_sampler_kwargs_final else None

        # --- Establecer Épocas para el Entrenamiento Final ---
        # Usamos FINAL_TRAINING_EPOCHS como máximo, EarlyStopping decidirá
        training_kwargs_final['num_epochs'] = FINAL_TRAINING_EPOCHS

        # Imprimir la configuración final que se pasará a pipeline
        print("\nConfiguración Final Reconstruida para pipeline:")
        print(f"  model={best_model_name_from_hpo}")
        print(f"  model_kwargs={model_kwargs_final}")
        print(f"  loss={loss_final}")
        print(f"  loss_kwargs={loss_kwargs_final}")
        print(f"  regularizer={regularizer_final}")
        print(f"  regularizer_kwargs={regularizer_kwargs_final if regularizer_final else 'None'}")
        print(f"  optimizer={optimizer_final}")
        print(f"  optimizer_kwargs={optimizer_kwargs_final}")
        print(f"  training_kwargs={training_kwargs_final}") # Contiene batch_size y num_epochs
        print(f"  negative_sampler={negative_sampler_final}")
        print(f"  negative_sampler_kwargs={negative_sampler_kwargs_final}")


        # --- Llamar a pipeline con argumentos reconstruidos ---
        best_pipeline_result = pipeline(
            training=training_factory,
            validation=validation_factory,
            testing=testing_factory,

            # --- Usar parámetros reconstruidos ---
            model=best_model_name_from_hpo,
            model_kwargs=model_kwargs_final,
            loss=loss_final,
            loss_kwargs=loss_kwargs_final,
            regularizer=regularizer_final,
            regularizer_kwargs=regularizer_kwargs_final if regularizer_final else None,
            optimizer=optimizer_final,
            optimizer_kwargs=optimizer_kwargs_final,
            training_kwargs=training_kwargs_final, # Incluye batch_size y num_epochs
            negative_sampler=negative_sampler_final,
            negative_sampler_kwargs=negative_sampler_kwargs_final,

            # Early Stopping para el entrenamiento final (ajusta paciencia/frecuencia si quieres)
            stopper='early',
            stopper_kwargs=dict(
                frequency=10, # Evaluar validación cada 10 épocas
                patience=10,  # Aumentar paciencia para entrenamiento final
                metric=HPO_METRIC,
                larger_is_better=(HPO_DIRECTION == 'maximize'),
            ),

            device=TARGET_DEVICE, # Usar GPU si está disponible
            random_seed=SEED, # Fijar semilla para reproducibilidad del entrenamiento final
            evaluation_kwargs=dict(batch_size=2048), # Batch grande para evaluación final
        )

        final_train_end_time = time.time()
        print(f"--- Entrenamiento Final Completado ---")
        print(f"Tiempo de Entrenamiento Final: {(final_train_end_time - final_train_start_time)/60:.2f} minutos")

        # Guardar el modelo final y sus resultados
        final_model_dir = os.path.join(RESULTS_DIR, 'final_best_model')
        # Asegurarse de que el directorio de modelo final exista
        os.makedirs(final_model_dir, exist_ok=True)
        best_pipeline_result.save_to_directory(final_model_dir)
        print(f"Modelo final y resultados guardados en: {final_model_dir}")

    except Exception as final_train_err:
        print(f"\nERROR FATAL durante el Entrenamiento Final: {type(final_train_err).__name__}: {final_train_err}")
        traceback.print_exc() # Imprimir traceback completo
        best_pipeline_result = None # Marcar que falló

# Manejar casos donde HPO falló o no se cargaron datos
elif 'load_successful' in locals() and load_successful:
    print("\n--- HPO falló o no encontró parámetros válidos. No se puede entrenar el modelo final. ---")
else:
    print("\n--- La carga/división de datos falló. Saltando Entrenamiento Final. ---")

# Actualizar flag para la celda de métricas basado en si pipeline_result existe
if best_pipeline_result is not None:
     try:
         # Intenta acceder a los resultados para confirmar que el objeto es válido
         _ = best_pipeline_result.metric_results.to_df()
         final_metrics_available = True
         print("Resultados del pipeline final disponibles.")
     except Exception as metric_err:
         print(f"Advertencia: El entrenamiento final pareció completarse, pero hubo un error al acceder a las métricas: {metric_err}")
         final_metrics_available = False
else:
     final_metrics_available = False

print("\n--- Fin de la Celda 6 (Entrenamiento Final) ---")

INFO:pykeen.pipeline.api:Using device: cuda
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /home/javimc/.data/pykeen/checkpoints/best-model-weights-c9f14675-5ae3-48e0-b5f7-f46ebb82a102.pt
INFO:pykeen.triples.triples_factory:Creating inverse triples.



--- Iniciando Entrenamiento Final con el Mejor Modelo y Parámetros ---
Modelo Seleccionado por HPO: TransE
Device para Entrenamiento Final: 'cuda'
Parámetros Optimizados a usar: {'model.embedding_dim': 384, 'model.scoring_fct_norm': 2, 'loss.margin': 1.8213771944268697, 'optimizer.lr': 0.0003184982126337148, 'negative_sampler.num_negs_per_pos': 6, 'training.num_epochs': 1000, 'training.batch_size': 512}

Configuración Final Reconstruida para pipeline:
  model=TransE
  model_kwargs={'embedding_dim': 384, 'scoring_fct_norm': 2}
  loss=MarginRankingLoss
  loss_kwargs={'margin': 1.8213771944268697}
  regularizer=None
  regularizer_kwargs=None
  optimizer=Adam
  optimizer_kwargs={'lr': 0.0003184982126337148}
  training_kwargs={'batch_size': 512, 'num_epochs': 200}
  negative_sampler=BasicNegativeSampler
  negative_sampler_kwargs={'num_negs_per_pos': 6}


Training epochs on cuda:0:   4%|▍         | 9/200 [00:22<07:01,  2.21s/epoch, loss=0.932, prev_loss=0.963]INFO:pykeen.evaluation.evaluator:Evaluation took 10.92s seconds
INFO:pykeen.stoppers.early_stopping:New best result at epoch 10: 0.5001839452578912. Saved model weights to /home/javimc/.data/pykeen/checkpoints/best-model-weights-c9f14675-5ae3-48e0-b5f7-f46ebb82a102.pt
INFO:pykeen.training.training_loop:=> Saved checkpoint after having finished epoch 10.
Training epochs on cuda:0:  55%|█████▍    | 109/200 [05:53<03:33,  2.35s/epoch, loss=0.85, prev_loss=0.853] INFO:pykeen.evaluation.evaluator:Evaluation took 10.98s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early at epoch 110. The best result 0.5001839452578912 occurred at epoch 10.
INFO:pykeen.stoppers.early_stopping:Re-loading weights from best epoch from /home/javimc/.data/pykeen/checkpoints/best-model-weights-c9f14675-5ae3-48e0-b5f7-f46ebb82a102.pt
Training epochs on cuda:0:  55%|█████▍    | 109/200 [06:04<05:04,  3.34

--- Entrenamiento Final Completado ---
Tiempo de Entrenamiento Final: 6.26 minutos


INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=19311, num_relations=16, create_inverse_triples=True, num_triples=108725) to file:///home/javimc/Desktop/KGE/pykeen_full_hpo_results/final_best_model/training_triples
INFO:pykeen.pipeline.api:Saved to directory: /home/javimc/Desktop/KGE/pykeen_full_hpo_results/final_best_model


Modelo final y resultados guardados en: pykeen_full_hpo_results/final_best_model
Resultados del pipeline final disponibles.

--- Fin de la Celda 6 (Entrenamiento Final) ---


In [16]:
# Celda 7: Mostrar Métricas del Modelo Final

# Reiniciar dataframe de resumen final (por si se re-ejecuta la celda)
final_summary_df = pd.DataFrame()
# Resetear flag (se establecerá si las métricas se procesan)
final_metrics_available = False

# Verificar si el entrenamiento final (Celda 6) fue exitoso y generó resultados
if 'best_pipeline_result' in locals() and best_pipeline_result is not None:
    print("\n--- Métricas del Modelo Final Entrenado (Test Set, Realistic, Tail) ---")
    try:
        # Obtener el DataFrame de métricas del resultado del pipeline
        final_metrics_df = best_pipeline_result.metric_results.to_df()

        # Filtrar usando las columnas correctas ('Side', 'Rank_type')
        # Asumiendo que estos son los resultados del test set evaluados por pipeline
        print(f"Intentando filtrar métricas con Side='tail' y Rank_type='realistic'...")
        final_test_metrics = final_metrics_df[
            (final_metrics_df['Side'] == 'tail') &
            (final_metrics_df['Rank_type'] == 'realistic')
        ]

        if not final_test_metrics.empty:
            # Imprimir las métricas clave (Metric y Value)
            print("Métricas Finales Obtenidas:")
            # Usar to_string para mejor formato si hay muchas métricas
            print(final_test_metrics[['Metric', 'Value']].round(4).to_string(index=False))

            # Guardar en un DataFrame de resumen (opcional, útil si comparas ejecuciones)
            # Obtener el nombre de la clase del modelo entrenado
            model_cls_name = best_pipeline_result.model.__class__.__name__
            final_summary_df[model_cls_name] = final_test_metrics.set_index('Metric')['Value']
            final_metrics_available = True # Marcar que tenemos métricas para inferencia
        else:
            print("ERROR: No se pudieron extraer las métricas finales con Side='tail' y Rank_type='realistic'.")
            print("Revisando todas las métricas disponibles en el DataFrame:")
            print(final_metrics_df.round(4).to_string()) # Imprimir todo para depurar

    except KeyError as filter_err:
        print(f"ERROR: KeyError al filtrar métricas finales: {filter_err}.")
        # Imprimir columnas disponibles si falla el filtrado
        if 'final_metrics_df' in locals():
             print(f"Columnas disponibles en métricas: {final_metrics_df.columns}")
        print("No se pueden mostrar las métricas finales.")
    except Exception as e:
        print(f"ERROR inesperado al procesar métricas finales: {type(e).__name__}: {e}")
        traceback.print_exc() # Imprimir traceback para errores inesperados

else:
    print("\n--- Entrenamiento Final (Celda 6) falló o fue saltado. No hay métricas finales que mostrar. ---")

# Mensaje final de la celda
if final_metrics_available:
    print("\n--- Métricas finales procesadas. Lista para inferencia (Celda 8). ---")
else:
    print("\n--- No se pudieron procesar las métricas finales. La inferencia (Celda 8) podría no funcionar o usar datos incorrectos. ---")

print("\n--- Fin de la Celda 7 (Métricas Finales) ---")


--- Métricas del Modelo Final Entrenado (Test Set, Realistic, Tail) ---
Intentando filtrar métricas con Side='tail' y Rank_type='realistic'...
Métricas Finales Obtenidas:
                             Metric      Value
       z_inverse_harmonic_mean_rank  7039.5872
                              count 13591.0000
                 standard_deviation   285.2331
 adjusted_geometric_mean_rank_index     0.9999
                        median_rank     2.0000
               arithmetic_mean_rank     6.7982
      adjusted_arithmetic_mean_rank     0.0007
          median_absolute_deviation     1.4826
                 harmonic_mean_rank     1.7957
             z_arithmetic_mean_rank   201.7915
         inverse_harmonic_mean_rank     0.5569
              z_geometric_mean_rank   116.7262
        inverse_geometric_mean_rank     0.4911
adjusted_inverse_harmonic_mean_rank     0.5566
                           variance 81357.9375
                inverse_median_rank     0.5000
adjusted_arithmetic_mean_rank

In [9]:
# Celda 8: Inferencia - Asumiendo 'tf' existe de Celda 4 (MODIFICADA)

import torch
import pandas as pd
import os
import traceback
# No necesitamos importar TriplesFactory aquí si asumimos que existe de Celda 4

# --- Preparación para Inferencia ---
print("\n--- Preparando Inferencia ---")
# Verificar que tf (de Celda 4) y el modelo final (guardado en Celda 6) existen
trained_model = None
load_inference_successful = False # Flag para saber si el modelo se cargó
final_model_dir = os.path.join(RESULTS_DIR, 'final_best_model')
model_path = os.path.join(final_model_dir, 'trained_model.pkl')

# Comprobar si las variables necesarias existen en memoria/disco
if 'tf' not in locals() or tf is None:
    print("ERROR: Objeto TriplesFactory 'tf' no encontrado en memoria.")
    print("       ASEGÚRATE DE HABER EJECUTADO la Celda 4 (Carga y División) en esta sesión.")
elif 'TARGET_DEVICE' not in locals():
     print("ERROR: TARGET_DEVICE no definido. Ejecuta Celda 3.")
elif not os.path.exists(model_path):
     print(f"ERROR: Archivo del modelo final no encontrado: {model_path}")
     print("       Asegúrate de que la Celda 6 se completó y guardó el modelo.")
else:
    # Intentar cargar solo el modelo desde el disco
    try:
        print(f"Cargando modelo desde: {model_path} hacia {TARGET_DEVICE}")
        trained_model = torch.load(model_path,
                                   map_location=torch.device(TARGET_DEVICE),
                                   weights_only=False
                                  )
        trained_model.eval() # Modo evaluación
        trained_model.to(TARGET_DEVICE) # Asegurar dispositivo
        print("Modelo cargado exitosamente.")
        load_inference_successful = True # Solo necesitamos cargar el modelo aquí
    except Exception as load_err:
        print(f"ERROR al cargar el modelo guardado: {type(load_err).__name__}: {load_err}")
        traceback.print_exc()

# --- Inferencia (Solo si el modelo se cargó y tf existe de Celda 4) ---
if load_inference_successful and trained_model is not None and tf is not None:
    device = TARGET_DEVICE
    print(f"\nModelo: {trained_model.__class__.__name__}, Device: {device}")
    # Usamos el 'tf' que ya está en memoria desde la Celda 4
    print(f"Usando TriplesFactory de Celda 4 (Entidades: {tf.num_entities}, Relaciones: {tf.num_relations})")
    print(f"Consulta: Ingrediente='{TARGET_INGREDIENT}', Calorías='{TARGET_CALORIE_LEVEL}'")

    recommendations_found = False
    ingredient_df = None
    calorie_df = None
    k_preds = tf.num_entities # Obtener scores para todas las entidades

    try:
        # --- Usar model.score_h y procesar manualmente ---

        # --- 1. Predicción por Ingrediente ---
        print("Realizando predicción por ingrediente (usando score_h)...")
        try:
            # Validar y obtener IDs desde 'tf' en memoria
            if INGREDIENT_RELATION not in tf.relation_to_id: raise KeyError(f"Relación '{INGREDIENT_RELATION}' no encontrada.")
            if TARGET_INGREDIENT not in tf.entity_to_id: raise KeyError(f"Entidad '{TARGET_INGREDIENT}' no encontrada.")
            rel_id_ing = tf.relation_to_id[INGREDIENT_RELATION]
            tail_id_ing = tf.entity_to_id[TARGET_INGREDIENT]
            # Crear batch (rel, tail)
            rt_batch_ing = torch.tensor([[rel_id_ing, tail_id_ing]], dtype=torch.long, device=device)
            # Calcular scores
            with torch.no_grad():
                head_scores_ing = trained_model.score_h(rt_batch_ing).squeeze(0)
            # Obtener top K
            top_scores_ing, top_indices_ing = torch.topk(head_scores_ing, k=k_preds, largest=True)
            # Mapear y crear DataFrame
            top_head_ids_ing = top_indices_ing.tolist()
            top_head_labels_ing = [tf.entity_id_to_label.get(idx, f"UNKNOWN_ID_{idx}") for idx in top_head_ids_ing]
            top_scores_list_ing = top_scores_ing.cpu().tolist()
            ingredient_df = pd.DataFrame({'head_label': top_head_labels_ing, 'score': top_scores_list_ing})
            print(f"-> Encontradas {len(ingredient_df)} recetas/entidades potenciales con '{TARGET_INGREDIENT}'.")
        except KeyError as e: print(f"Error Mapeo (Ingrediente): {e}")
        except Exception as e_ing: print(f"Error Pred. Ingrediente: {type(e_ing).__name__}: {e_ing}"); traceback.print_exc()

        # --- 2. Predicción por Calorías ---
        if ingredient_df is not None: # Continuar solo si el paso anterior no falló catastróficamente
            print("Realizando predicción por nivel calórico (usando score_h)...")
            try:
                # Validar y obtener IDs desde 'tf' en memoria
                if CALORIE_RELATION not in tf.relation_to_id: raise KeyError(f"Relación '{CALORIE_RELATION}' no encontrada.")
                if TARGET_CALORIE_LEVEL not in tf.entity_to_id: raise KeyError(f"Entidad '{TARGET_CALORIE_LEVEL}' no encontrada.")
                rel_id_cal = tf.relation_to_id[CALORIE_RELATION]
                tail_id_cal = tf.entity_to_id[TARGET_CALORIE_LEVEL]
                # Crear batch (rel, tail)
                rt_batch_cal = torch.tensor([[rel_id_cal, tail_id_cal]], dtype=torch.long, device=device)
                # Calcular scores
                with torch.no_grad():
                    head_scores_cal = trained_model.score_h(rt_batch_cal).squeeze(0)
                # Obtener top K
                top_scores_cal, top_indices_cal = torch.topk(head_scores_cal, k=k_preds, largest=True)
                # Mapear y crear DataFrame
                top_head_ids_cal = top_indices_cal.tolist()
                top_head_labels_cal = [tf.entity_id_to_label.get(idx, f"UNKNOWN_ID_{idx}") for idx in top_head_ids_cal]
                top_scores_list_cal = top_scores_cal.cpu().tolist()
                calorie_df = pd.DataFrame({'head_label': top_head_labels_cal, 'score': top_scores_list_cal})
                print(f"-> Encontradas {len(calorie_df)} recetas/entidades potenciales con '{TARGET_CALORIE_LEVEL}'.")
            except KeyError as e: print(f"Error Mapeo (Calorías): {e}")
            except Exception as e_cal: print(f"Error Pred. Calorías: {type(e_cal).__name__}: {e_cal}"); traceback.print_exc()

        # --- 3. Combinar Resultados ---
        print("Combinando y rankeando resultados...")
        if ingredient_df is not None and not ingredient_df.empty and calorie_df is not None and not calorie_df.empty:
            ingredient_df = ingredient_df.rename(columns={'score': 'ingredient_score'})
            calorie_df = calorie_df.rename(columns={'score': 'calorie_score'})
            recommendations = pd.merge(
                ingredient_df[['head_label', 'ingredient_score']],
                calorie_df[['head_label', 'calorie_score']],
                on='head_label', how='inner'
            )
            if not recommendations.empty:
                recommendations['combined_score'] = recommendations['ingredient_score'] + recommendations['calorie_score']
                recommendations = recommendations.sort_values(by='combined_score', ascending=False).head(20)
                print(f"\n--- Recomendaciones Finales (Top {len(recommendations)}) ---")
                print(recommendations[['head_label', 'combined_score']].round(4).to_string(index=False))
                recommendations_found = True
            else: print(f"\n-> No se encontraron recetas comunes después del merge.")
        else: print("\n-> No se pueden combinar: uno o ambos DataFrames iniciales están vacíos o fallaron.")

    except Exception as infer_err:
        print(f"\nError inesperado durante el proceso de inferencia: {type(infer_err).__name__}: {infer_err}")
        traceback.print_exc()

    if not recommendations_found:
        print(f"\n-> No se pudieron generar recomendaciones combinadas finales para '{TARGET_INGREDIENT}' y '{TARGET_CALORIE_LEVEL}'.")

else:
     print("\n--- Saltando Inferencia: Falló la carga del modelo o el TriplesFactory 'tf' no está disponible en memoria. ---")
     print("--- Asegúrate de ejecutar las Celdas 1, 2, 3 y 4 antes de esta celda. ---")


print("\n--- Fin de la Celda 8 (Inferencia) ---")


--- Preparando Inferencia ---
Cargando modelo desde: pykeen_full_hpo_results/final_best_model/trained_model.pkl hacia cuda
Modelo cargado exitosamente.

Modelo: TransE, Device: cuda
Usando TriplesFactory de Celda 4 (Entidades: 19311, Relaciones: 16)
Consulta: Ingrediente='potato', Calorías='high_calories'
Realizando predicción por ingrediente (usando score_h)...
-> Encontradas 19311 recetas/entidades potenciales con 'potato'.
Realizando predicción por nivel calórico (usando score_h)...
-> Encontradas 19311 recetas/entidades potenciales con 'high_calories'.
Combinando y rankeando resultados...

--- Recomendaciones Finales (Top 20) ---
            head_label  combined_score
          quickandeasy         -5.6961
           baconandegg         -5.7103
          sweetandsour         -5.7114
            worldsbest         -5.7157
            pauladeens         -5.7160
              todiefor         -5.7188
      creamychickenand         -5.7197
roastedbutternutsquash         -5.7212
      